![alt text](https://anvaqta.id/headerai.jpg "Compression.dl")

# Import Library

Lakukan import library yang akan digunakan selama proses klasifikasi artikel.

In [0]:
!pip install tensorflow-gpu==2.0.0

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import model_from_json, load_model

# Lakukan import fungsi train_test_split dari library sklearn.model_selection
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import re # regex

# Preprocessing Data Train

Hal pertama yang harus dilakukan adalah membaca atau menyimpan data yang akan digunakan dalam proses pembelajaran klasifikasi.

In [0]:
# Lakukan read data 'Data Training.xlsx' menggunakan fungsi read_excel pada library pandas
# Beri nama kolom 0 dengan 'artikel' dan kolom 1 dengan 'label'
data = ??

Mari kita lihat data train yang akan kita gunakan!
Dapat kita lihat pada data train yang baru saja disimpan masih mengandung huruf kapital dan karakter khusus ('.', '!', dll).

In [0]:
data.head(5)

Setelah itu, lakukan pembersihan data train dengan mengubah seluruh huruf ke dalam huruf kecil dan membersihkannya dari karakter khusus

In [0]:
regex = r'[^a-z\s]'

# Ubah seluruh huruf dalam data artikel ke dalam lower case
data['artikel'] = ??

# Hapus seluruh simbol yang ada dan ganti dengan string kosong ('')
data['artikel'] = ??

Mari bandingkan data yang telah dilakukan preprocessing dengan data yang masih mentah tadi.

In [0]:
data.head(5)

Lakukan tokenisasi terhadap seluruh data set yang telah dibersihkan dengan menggunakan tokenizer yang dimiliki oleh tensorflow keras.

In [0]:
# Tentukan jumlah max_features (jumlah kata yang ingin digunakan pada setiap artikel)
max_features = 500

# Buat object Tokenizer() dari keras, 
# gunakan parameter num_words untuk membatasi kata yang ingin ditokenisasi pada tiap artikel
# dan parameter split untuk menentukan pembatas tiap kata, (' ') = spasi
tokenizer = Tokenizer(num_words=max_features, split=' ')

# Lakukan tokenisasi pada setiap artikel dengan tokenizer yang telah dibuat
tokenizer.fit_on_texts(data['artikel'].values)

# Ubah token menjadi sequence/angka-angka
X = tokenizer.texts_to_sequences(data['artikel'].values)

# Lakukan padding, jika jumlah kata dalam artikel tidak sebanyak max_features maka diisi dengan angka (0)
X = pad_sequences(X)

# Pendefinisan Label

Definisikan seluruh label artikel ke dalam sebuah list

In [0]:
# Tentukan label yang digunakan
labels = [??]

Kemudian cek jumlah artikel yang terdapat pada setiap label untuk melihat data yang dimiliki bersifat imbalance atau tidak.
Jika terlihat data yang memiliki jumlah 0, maka penamaan label kemungkinan ada yang salah

In [0]:
for label in labels:
    print(label)
    print(data[data['label']==label].shape[0])

# Pembangunan Model

![alt text](https://indoml.files.wordpress.com/2018/03/kind-many-to-one2.jpg)

Langkah selanjutnya ialah membangun model klasifikasi dengan menggunakan metode LSTM

In [0]:
embed_dim = 128
lstm_out = 196

# Inisiasi model dengan sequential
model = Sequential()
# menambahkan model dengan layer Embedding dengan input_dim sebesar max_features, output_dim sebesar embed_dim, dan input_length sebesar panjang X
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
# menambahkan model dengan layer SpatialDropout1D sebesar 0.4
model.add(SpatialDropout1D(0.4))
# menambahkan model dengan LSTM sebanyak lstm_out, dengan dropout dan recurrent_dropout sebesar 0.5, dan parameter return_sequences True
model.add(LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.5, return_sequences=True))
# menambahkan model dengan LSTM sebanyak lstm_out, dengan dropout dan recurrent_dropout sebesar 0.5, tanpa parameter return_sequences
model.add(LSTM(lstm_out, dropout=0.5, recurrent_dropout=0.5))
# menambahkan model dengan dense sebanyak jumlah label dan aktivasi menggunakan softmax
model.add(Dense(5,activation='softmax'))
# model akan di compile menggunakan parameter loss 'categorical_crossentropy', optimizer 'adam' dan metrics accuracy
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

In [0]:
# Menampilkan layer yang telah dibuat
from tensorflow.keras.utils import plot_model
plot_model(model)

Untuk dapat menguji model yang telah dibangun, maka lakukan split (pembagian) data train menjadi dua data berbeda, yaitu data train dan data validasi. 

Gunakan data validasi sebesar 33% dari data yang dimiliki.

In [0]:
# Ubah data label menjadi variable dummy dengan menggunakan library pandas (pd.get_dummies())
Y = ??
# Lakukan split data menggunakan function train_test_split() pada sklearn 
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)

Latih model dengan data train yang telah displit

In [0]:
batch_size = 32
history = model.fit(X_train, Y_train,
                    validation_data=(X_val, Y_val),
                    epochs = 30, batch_size=batch_size, verbose = 1)

In [0]:
# Menampilkan grafik accuracy dan validation accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

# Tampilkan grafik yang telah diplot
plt.show()

In [0]:
# Menampilkan hasil evaluasi akhir
score,acc = model.evaluate(X_val, Y_val, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

# Pengujian

Input kalimat yang akan diuji dengan model yang telah dibangun

In [0]:
#@title Input Kalimat
#@markdown Kalimat diawali dan diakhiri tanda kutip (" ")

kalimat= "  "  #@param
#@markdown ---


Lakukan pembersihan data terhadap inputan kalimat

In [0]:
# Ubah hasil inputan kalimat ke dalam bentuk lower case
kalimat = ??

# Hapus seluruh simbol dan ganti dengan string kosong (''), gunakan function re.sub
kalimat = ??

# Ubah hasil inputan kedalam list/array
kalimat = [kalimat]

print(kalimat)

Jangan lupa lakukan tokenisasi terhadap inputan kalimat

In [0]:
# Ubah token menjadi sequence/angka2
test_kalimat = tokenizer.texts_to_sequences(kalimat)

# Lakukan padding, jika jumlah kata dalam artikel tidak sebanyak max_features maka diisi dengan angka (0)
test_kalimat = pad_sequences(test, maxlen=X.shape[1])

Ayo prediksi label dari inputan kalimat!

In [0]:
# Ubah kalimat yang ingin diuji kedalam bentuk vector
vector = np.array([test.flatten()])

# Prediksi label dari kalimat tersebut menggunakan function model.predict_classes()
y = ??

print(labels[y[0]])

# Penyimpanan Model

Simpan model ke dalam file dengan format .h5

In [0]:
model.save_model("model.h5")

Unduh model

In [0]:
from google.colab import files
files.download('model.h5')

Gunakan code di bawah untuk memuat ulang model (load model)

In [0]:
# model = load_model("model.h5")